Gcode 41

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 


In [2]:
model = AbstractModel()
model.gen=Set()
model.t=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.d=Param(model.gen)
model.e=Param(model.gen)
model.f=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)
model.RU=Param(model.gen)
model.RD=Param(model.gen)
model.demand=Param(model.t)
#Scal

In [3]:
#constraint
def Pbounds(model,gen,t):
  return (model.Pmin[gen],model.Pmax[gen])

model.p=Var(model.gen,model.t,bounds=Pbounds,within=Reals)
model.cost=Var(within=Reals)
model.EM=Var(within=Reals)

def rule1(model,t):
  return sum(model.p[gen,t] for gen in model.gen) >= model.demand[t]
model.c1 = Constraint(model.t,rule=rule1)

def rule2(model,gen,t):
    if t==1:
      return model.p[gen,t+1] - model.p[gen,t] <= model.RU
    else:
     return Constraint.Skip
model.c2 = Constraint(model.gen,model.t,rule=rule2)

def rule3(model,gen,t):
    if t==1:
      return model.p[gen,t-1]- model.p[gen,t] <= model.RD
    else:
        return Constraint.Skip
model.c3 = Constraint(model.gen,model.t,rule=rule3)

def ruleTE(model):
  return model.EM == sum((model.d[gen]*model.p[gen,t]*model.p[gen,t] + 
                              model.e[gen]*model.p[gen,t]+model.f[gen]) for gen in model.gen for t in model.t) 
model.c5 = Constraint(rule=ruleTE)

def ruleTC(model):
  return model.cost == sum((model.a[gen]*model.p[gen,t]*model.p[gen,t] + 
                              model.b[gen]*model.p[gen,t]+model.c[gen]) for gen in model.gen for t in model.t) 
model.c4 = Constraint(rule=ruleTC)



In [4]:
model.obj=Objective(expr=model.cost,sense=minimize)

In [5]:
instance= model.create_instance("code41.dat")

In [6]:
results = SolverFactory('ipopt').solve(instance).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 98
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.14.10\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.05581307411193848
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [7]:
print('Total Cost  =   ',round(instance.cost(),2));
print('Total Emission=  ',round(instance.EM(),2));

Total Cost  =    638553.24
Total Emission=   3535979.14


In [8]:
instance.pprint()

2 Set Declarations
    gen : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'G1', 'G2', 'G3', 'G4'}
    t : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24'}

11 Param Declarations
    Pmax : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :   200
         G2 :   290
         G3 :   190
         G4 :   260
    Pmin : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    28
         G2 :    20
         G3 :    30
         G4 :    20
    RD : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    40
         G2 :    30
         G3 :    30
         G4 :    50

In [22]:
instance.p.pprint()

p : Size=96, Index=gen*t
    Key           : Lower : Value              : Upper : Fixed : Stale : Domain
     ('G1', 't1') :    28 : 102.44473948947329 :   200 : False : False :  Reals
    ('G1', 't10') :    28 :  200.0000019998365 :   200 : False : False :  Reals
    ('G1', 't11') :    28 : 200.00000199985953 :   200 : False : False :  Reals
    ('G1', 't12') :    28 : 200.00000199987466 :   200 : False : False :  Reals
    ('G1', 't13') :    28 : 200.00000199986752 :   200 : False : False :  Reals
    ('G1', 't14') :    28 : 200.00000199973405 :   200 : False : False :  Reals
    ('G1', 't15') :    28 : 200.00000199968082 :   200 : False : False :  Reals
    ('G1', 't16') :    28 : 200.00000199980448 :   200 : False : False :  Reals
    ('G1', 't17') :    28 : 200.00000199978655 :   200 : False : False :  Reals
    ('G1', 't18') :    28 :  200.0000019998758 :   200 : False : False :  Reals
    ('G1', 't19') :    28 : 200.00000199982193 :   200 : False : False :  Reals
     ('G1', 't2